In [1]:
import autodiff.admath.admath as math
from autodiff.interface.interface import AutoDiff as AD

In [2]:
def square_fn(x):
    return x ** 2

ad_square = AD(square_fn)
ad_square.get_der(3)

6

In [3]:
ad_square.get_der([1,2])

[2, 4]

In [4]:
def sin_fn(x):
    return math.sin(x)

ad_sin = AD(sin_fn)
ad_sin.get_der(0)

1.0